In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
url='https://rollcall.com/factbase/trump/transcript/donald-trump-speech-religious-liberty-commission-september-8-2025/'
resp=requests.get(url)
soup=BeautifulSoup(resp.text,'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://rollcall.com/app/themes/rollcall/public/favicon.dfdc32.ico" rel="icon" type="image/x-icon"/>
<meta content="#2F3B4B" name="msapplication-TileColor"/>
<link href="https://rollcall.com/app/themes/rollcall/public/pwa-manifest.json" rel="manifest"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<style>img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }</style>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="#2F3B4B" name="theme-color"/>
<meta content="#2F3B4B" name="msapplication-TileColor"/>
<meta content="https://rollcall.com/app/themes/rollcall/app/uploads/2022/02/cropped-rollcall-placeholder-

Title

In [4]:
speech_title=soup.find(class_="text-[#2F3C4B] text-center text-xl sm:text-2xl not-italic font-semibold leading-normal sm:leading-9 font-graphik")
speech_titles=speech_title.get_text(strip=True)
speech_title=re.search(r"^(.+)-",speech_titles).group(1).strip()
print(speech_titles)
speech_date=re.search(r"-\s(.+)",speech_titles).group(1).strip()
speech_date


Speech: Donald Trump Addresses the Religious Liberty Commission in Washington - September 8, 2025


'September 8, 2025'

Number of sentences and number of words, all in one, strong

In [5]:
blocks_container=soup.find(class_="flex flex-wrap gap-8 justify-between")
blocks=blocks_container.find_all(class_='flex-1 h-content')
for block in blocks:
    name_div=block.find(class_="font-graphik text-sm font-medium leading-normal flex items-center").get_text(strip=True)
    if "Trump" in name_div:
        trump_block=block
contents=trump_block.find_all(class_="font-graphik text-xs font-medium text-[#2F3C4B]")
contents=[content.get_text(strip=True) for content in contents]
contents
for content in contents :
    if "sentences" in content:
        nbr_sentences=re.search(r'\d+',content).group()
    elif "words" in content:
        nbr_words=re.search(r'\d+',content).group()
    else :
        nbr_seconds=re.search(r'\d+',content).group()
print(nbr_sentences,nbr_words,nbr_seconds)

703 7094 2588


Categories

In [6]:
categories=soup.find_all(class_="text-[#015582] text-sm font-normal leading-normal rounded-md bg-[#F4F4F5] border border-[#D9D9D9] p-2")
categories=[category.get_text(strip=True) for category in categories]



Cleaning Categories

In [7]:
categories=[category.split('>') for category in categories]
categories=list(set([category[i].strip() for category in categories for i in range(len(category))]))
categories

['Court',
 'Belief Systems',
 'Politics',
 'Judiciary',
 'Christianity',
 'Religion',
 'Crime, Law and Justice']

Transcriptions

In [8]:
transcriptions=soup.find_all(class_='flex gap-4 py-2')
list_transcriptions=[]
for transcription in transcriptions :
    speaker=transcription.find(class_="text-md inline").get_text(strip=True)
    timestamp=transcription.find(class_='text-xs text-gray-600 inline ml-2').get_text(strip=True)
    text=transcription.find(class_='flex-auto text-md text-gray-600 leading-loose').get_text(strip=True)
    list_transcriptions.append([speaker,timestamp,text])
list_transcriptions

[['Donald Trump',
  '00:00:00-00:00:22 (23 sec)',
  "Well, thank you very much. Thank you very much. I said we'll have to cut Lee Greenwood a little bit short. How good is he? How good is he, right? He's been with us right from the beginning and I wanted to stand and listen to the entire song. But people would have said, look at the ego of that guy, he just wouldn't stop. [Laughter] He just stood there and listened to the applause."],
 ['Donald Trump',
  '00:00:22-00:00:46 (24 sec)',
  'But I appreciate it very much, and I want to thank you for doing an incredible job, Pam Bondi. The attorney general has been unbelievable. Incredible job. Thank you, Pam. Amazing. And thank you to all of the faith leaders, pastors and patriots who have come to the Museum of the Bible for this historic meeting of the Presidential Commission on Religious Liberty.'],
 ['Donald Trump',
  '00:00:46-00:01:10 (24 sec)',
  "These are incredible people, and everybody wanted to be on this commission, and I made a

trump_transcriptions

In [9]:
trump_transcriptions=[transcription_list[1:] for transcription_list in list_transcriptions if "Donald Trump" in transcription_list[0]]
trump_transcriptions


[['00:00:00-00:00:22 (23 sec)',
  "Well, thank you very much. Thank you very much. I said we'll have to cut Lee Greenwood a little bit short. How good is he? How good is he, right? He's been with us right from the beginning and I wanted to stand and listen to the entire song. But people would have said, look at the ego of that guy, he just wouldn't stop. [Laughter] He just stood there and listened to the applause."],
 ['00:00:22-00:00:46 (24 sec)',
  'But I appreciate it very much, and I want to thank you for doing an incredible job, Pam Bondi. The attorney general has been unbelievable. Incredible job. Thank you, Pam. Amazing. And thank you to all of the faith leaders, pastors and patriots who have come to the Museum of the Bible for this historic meeting of the Presidential Commission on Religious Liberty.'],
 ['00:00:46-00:01:10 (24 sec)',
  "These are incredible people, and everybody wanted to be on this commission, and I made a couple of people unhappy -- maybe even enemies for li

Création de la première row de la DB

In [10]:
import pandas as pd

df=pd.DataFrame(columns=["Date","Title","Categories","Nbr Sentences","Nbr Words", "Nbr Seconds", "Transcriptions_list"])
df.loc[len(df)]=[speech_date,speech_title,categories,nbr_sentences,nbr_words,nbr_seconds,trump_transcriptions]
df

,Date,Title,Categories,Nbr Sentences,Nbr Words,Nbr Seconds,Transcriptions_list
0,"September 8, 2025",Speech: Donald Trump Addresses the Religious L...,"[Court, Belief Systems, Politics, Judiciary, C...",703,7094,2588,"[[00:00:00-00:00:22 (23 sec), Well, thank you ..."


In [14]:
df.iloc[0].loc['Transcriptions_list']

[['00:00:00-00:00:22 (23 sec)',
  "Well, thank you very much. Thank you very much. I said we'll have to cut Lee Greenwood a little bit short. How good is he? How good is he, right? He's been with us right from the beginning and I wanted to stand and listen to the entire song. But people would have said, look at the ego of that guy, he just wouldn't stop. [Laughter] He just stood there and listened to the applause."],
 ['00:00:22-00:00:46 (24 sec)',
  'But I appreciate it very much, and I want to thank you for doing an incredible job, Pam Bondi. The attorney general has been unbelievable. Incredible job. Thank you, Pam. Amazing. And thank you to all of the faith leaders, pastors and patriots who have come to the Museum of the Bible for this historic meeting of the Presidential Commission on Religious Liberty.'],
 ['00:00:46-00:01:10 (24 sec)',
  "These are incredible people, and everybody wanted to be on this commission, and I made a couple of people unhappy -- maybe even enemies for li